# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 데이터 결측치 처리하기
- 파생변수 생성하기

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [52]:
# basic
import os, random
import pandas as pd
import numpy as np

# graph
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# imputator
from sklearn.impute import KNNImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 폰트
plt.rcParams['font.family'] = 'NanumSquare'

# 마이너스 출력
matplotlib.rcParams['axes.unicode_minus'] = False

In [53]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [54]:
# load data
train = pd.read_csv('../data/fog_train.csv')
test = pd.read_csv('../data/fog_test.csv')

In [55]:
# 쓸데없는 열 제거
train.drop(['Unnamed: 0'], axis = 1, inplace = True)
test.drop(['Unnamed: 0'], axis = 1, inplace = True)


### 3. 데이터 결측치 처리하기

#### 3-1) 없는 날짜

현재 데이터에 존재하지 않는 레이블이 21개가 존재한다.  
- 지역별로 I년도 01월 01일 00:00분이 없기 때문에 공통적으로 없는 부분은 채우지 않고 그대로 진행하기로 한다.
- 그러나 중간에 없는 EC지역의 J년 11월 3일 18:20분은 직접 채우기로 한다.

In [56]:
# train 없는 날짜 채우기
train = train.append([{'fog_train.year':'J',
                'fog_train.month':11, 
                'fog_train.day':3,
                'fog_train.time':18, 
                'fog_train.minute':20,
                'fog_train.stn_id':'EC'}], ignore_index=True, sort=True)

In [57]:
# 결측치 nan으로 바꾸기
train2 = train.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)
test2 = test.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)

In [58]:
# 변수 이름 앞에있는 이상한거 제거하기
train2.columns = train2.columns.str.replace('fog_train.', '')
test2.columns = test2.columns.str.replace('fog_test.', '')

#### 3-2) 지역의 유형별로 나누기

In [59]:
# A:내륙, B:내륙산간, C:동해, D:서해, E:남해
train2['ground'] = train2['stn_id'].str.slice(0, 1)
test2['ground'] = test2['stn_id'].str.slice(0, 1)

#### 3-3) 연도를 적합하기 좋은 연도로 바꿔주기
- 그래프로 그려보기 편하게 I -> 2020 ~ K -> 2022
- 빠르게 바꾸도록 하자

In [60]:
# 시간
train2['hour'] = train2['time']

# I, J, K -> 2020 ~ 2022
cri = [
    train2['year'] == "I",
    train2['year'] == "J"
]
con = [
    2020,
    2021
]
train2['yeartmp'] = np.select(cri, con, default = 2022)

# year 값 바꾸기
train3 = train2.copy()
train3['year'] = train3['yeartmp']
train3.drop(['yeartmp'], axis = 1, inplace = True)

# 연, 월, 일, 시간, 분을 하나의 datetime 객체로 변환하고 문자열 형식으로 변환
train3['DateTime'] = pd.to_datetime(train3[['year', 'month', 'day', 'hour', 'minute']]).dt.strftime('%Y-%m-%d %H:%M')

#### 3-4) 이상치 대치

- 클래스가 뾰족하게 4 -> 1 -> 4 와 같이 분포하고 있는 것들이 몇개 있다.
- 이를 보완하기 위해 다음과 같은 과정을 진행한다.
- 단, 1000 이하를 중심으로 맞춰야하는 대회이기 때문에 class를 기준으로 적용하였다.

In [61]:
# 지역별로 따로따로 적용하자.
for c in train3['stn_id'].unique():

    # 지역별로 잘라서 이상치 -> 결측치 만들기
    tmp = train3[train3['stn_id'] == c]

    # lag 만들어서 지난번  class 확인하기
    tmp['shift_left_class'] = tmp['class'].shift(1)

    # 이전과의 차이 구하기
    tmp['diff_left_class'] = tmp['class'] - tmp['shift_left_class']

    # lag 만들어서 이다음 class 확인하기
    tmp['shift_right_class'] = tmp['class'].shift(-1)

    # 이다음과의 차이 구하기
    tmp['diff_right_class'] = tmp['class'] - tmp['shift_right_class']

    # 인덱스 뽑기
    idx = tmp[(abs(tmp['diff_left_class']) >= 2) & (abs(tmp['diff_right_class']) >= 2)].index

    # 인덱스 중 가만히 둬야할 인덱스 제외
    new_idx = []
    for i in idx:
        if i-1 not in idx:
            new_idx.append(i)

    # 인덱스로 처리
    new_idx = pd.Index(new_idx)

    # 뾰족한 데이터 결측치로 처리하기
    train3['class'].iloc[new_idx] = tmp['shift_left_class'][new_idx]
    train3['vis1'].iloc[new_idx] = train3['vis1'][new_idx-1]

#### 3-5) hm

- 습도가 완전히 0이 되는 경우는 존재할 수 없다.
- 또한 train에서만 hm이 0인 경우가 1건 존재하였기 때문에 train에서만 0의 값을 과거의 습도와 미래의 습도로 치환해주도록 한다.
- 혹여나 test에 습도가 0인 경우가 존재하면 nan값을 적용하여 knn 적용시 채우도록 적용한다.

In [62]:
# hm이 완전 0인경우 0.00001 더하자
train3['hm'][train3['hm'] == 0.0] = 66.3
test2['hm'][test2['hm'] == 0.0] = np.nan

#### 3-6) 시간적 사이클

- 계절에 따른 차이가 존재할 수 있기 때문에 계절에 관한 사이클을 만들어 주도록 하자
- 하루 단위로 온도의 변화가 존재하기 때문에 이를 학습시키기 위해 하루 단위를 변화하도록 하자

In [63]:
# 시간 사이클 변수
train3['sin_time'] = np.sin(2 * np.pi * train3['time'] / 24)
train3['cos_time'] = np.cos(2 * np.pi * train3['time'] / 24)

test2['sin_time'] = np.sin(2 * np.pi * test2['time'] / 24)
test2['cos_time'] = np.cos(2 * np.pi * test2['time'] / 24)

In [64]:
# 계절 사이클 변수 - 월별 주기
train3['sin_month'] = np.sin(2 * np.pi * train3['month'] / 12)
train3['cos_month'] = np.cos(2 * np.pi * train3['month'] / 12)

test2['sin_month'] = np.sin(2 * np.pi * test2['month'] / 12)
test2['cos_month'] = np.cos(2 * np.pi * test2['month'] / 12)

#### 3-7) KNN Imputation

- 서로간의 근접 이웃을 통해 값을 유사하게 채워주는 보간법이다.
- 내륙, 해안, 산간지방의 5가지 유형을 통해 분해하여 따로 처리해주는 방식을 사용한다.

- 다음과 같은 방법으로 결측치를 채우도록 하자

> 1. vis1을 제외한 train으로 knnimputator을 적용한 다음 test에 유형에 맞춰 값을 대치한다.
>    - 주의할점 : 시간에 관련된 변수는 제외한다. 값을 반영함으로 오히려 방해될 수 있다.
>    - 대신 삼각 치환을 통한 변수를 대신 대입하도록 한다.
> 2. 대치된 값을 활용하여 vis1을 만들고 class를 채운다.
>    - vis1의 변수 조절을 위해 log1p와 expm1 변환을 통해 조절해준다.

In [65]:
# log1p
train3['vis1'] = np.log1p(train3['vis1'])

In [66]:
# imputate with knnimputer
for c in ['E', 'B', 'C', 'D', 'A']:
    
    print(f'The first processing: {c} ground transforming ...')
    # 1. train & test knn imputer
    knnimputer = KNNImputer()

    # train - fit_transform
    print("train transforming ....")
    train3.loc[train3['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'sin_time', 'cos_time', 'cos_month', 'sin_month']] = knnimputer.fit_transform(train3[train3['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'sin_time', 'cos_time', 'cos_month', 'sin_month']])

    # test - transform
    print("test transforming ....")
    test2.loc[test2['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'sin_time', 'cos_time', 'cos_month', 'sin_month']] = knnimputer.transform(test2[test2['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'sin_time', 'cos_time', 'cos_month', 'sin_month']])
    
    # 2. vis1 imputater
    visimputer = KNNImputer()

    print(f'The second processing: {c} vis1 transforming ...')
    # only train
    train3.loc[train3['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'vis1', 'sin_time', 'cos_time', 'cos_month', 'sin_month']] = knnimputer.fit_transform(train3[train3['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'vis1', 'sin_time', 'cos_time', 'cos_month', 'sin_month']])
    print(f'The end of {c} ground imputate. check this train data')

The first processing: E ground transforming ...
train transforming ....
test transforming ....
The second processing: E vis1 transforming ...
The end of E ground imputate. check this train data
The first processing: B ground transforming ...
train transforming ....
test transforming ....
The second processing: B vis1 transforming ...
The end of B ground imputate. check this train data
The first processing: C ground transforming ...
train transforming ....
test transforming ....
The second processing: C vis1 transforming ...
The end of C ground imputate. check this train data
The first processing: D ground transforming ...
train transforming ....
test transforming ....
The second processing: D vis1 transforming ...
The end of D ground imputate. check this train data
The first processing: A ground transforming ...
train transforming ....
test transforming ....
The second processing: A vis1 transforming ...
The end of A ground imputate. check this train data


In [67]:
# expm1
train3['vis1'] = np.expm1(train3['vis1'])

#### 3-8) class 계산하기

- vis1을 활용하면 class가 계산된다.
- 실제로 기준의 범위를 나눌때 vis1 변수를 활용하기 때문에 이번에도 이와 같이 반영해준다.

In [68]:
# vis1을 활용하여 class 계산해주기
# 기준
cri = [
    (train3['class'].isna()) & (0 < train3['vis1']) & (train3['vis1'] < 200),
    (train3['class'].isna()) & (200 <= train3['vis1']) & (train3['vis1'] < 500),
    (train3['class'].isna()) & (500 <= train3['vis1']) & (train3['vis1'] < 1000),
    (train3['class'].isna()) & (1000 <= train3['vis1'])
]

# 반영값
con = [
    1, 2, 3, 4
]

# train
train3['class'] = np.select(cri, con, default = train3['class'])

#### 3-9) 후보정 처리
- 강수 여부, class와 같은 변수의 경우 정수로 입력되는 값을 받아야 할것이다. 따라서 후보정 처리가 들어가야 하는 변수를 조절해준다.
- 방향과 같은 경우 0 ~ 360 사이로 입력되게 만들어주어야 한다.

In [69]:
# 강수여부 처리
train3['re'] = np.where(train3['re'] >= 0.5, 1, 0)
test2['re'] = np.where(test2['re'] >= 0.5, 1, 0)

In [70]:
# 클래스 데이터 형태 변환
train3['class'] = train3['class'].astype(int)
# test2['class'] = test2['class'].astype(int)

### 4. 파생변수 생성하기

#### 4-1) 이슬점 온도

- 안개 생기는 기준점을 미리 만들어두자
- 이슬점은 지면온도와 기온을 고려했을 때 안개가 생성되게 하는 가장 좋은 기준점이다.

In [71]:
# Magnus 공식 상수
a = 17.27
b = 237.7

# 알파 값 계산
train3['alpha'] = (a * train3['ta']) / (b + train3['ta']) + np.log(train3['hm'] / 100.0)
test2['alpha'] = (a * test2['ta']) / (b + test2['ta']) + np.log(test2['hm'] / 100.0)

# 이슬점온도 계산
train3['dew_point'] = (b * train3['alpha']) / (a - train3['alpha'])
test2['dew_point'] = (b * test2['alpha']) / (a - test2['alpha'])

#### 4-2) 안개 발생 조건
  
- 지면 온도 - 이슬점 온도  
- 기온 - 이슬점 온도  

특징
- 기온이나 지면이 이슬점온도보다 낮아지게 되면 안개가 더 잘 발생한다고 알려져 있다.  
- 또한 온도의 차이가 낮아질 수록 안개 발생시 안개 농도 정도가 더 진해진다고 한다.  

In [72]:
# 온도조건 미리 계산하기
train3['diff_air-dew'] = train3['ta'] - train3['dew_point']
train3['diff_ts-dew'] = train3['ts'] - train3['dew_point']

test2['diff_air-dew'] = test2['ta'] - test2['dew_point']
test2['diff_ts-dew'] = test2['ts'] - test2['dew_point']

#### 4-3) AWS안개 생성 위험군 분류 지표

- AWS에서 사용하고 있는 기준을 가져와 우리 데이터에 맞게 적용시켜보고자 함
- 실제로 사용되고 있는 방법인 만큼 큰 효과 있기를 기대하며 생성하는 변수

In [73]:
# 안개 위험군 분류 지표
# train
cri = [
    # 5단계: high risk
    (train3['hm'] >= 97) & (train3['ws10_ms'] <= 7) & (train3['re'] == 0),

    # 4단계: middle risk
    (train3['hm'] < 97) & (train3['hm'] >= 95) & (train3['ws10_ms'] <= 7) & (train3['re'] == 0),

    # 3단계: Low risk
    (train3['hm'] < 95) & (train3['hm'] >= 90) & (train3['ws10_ms'] <= 7) & (train3['re'] == 0),

    # 2단계: Risk not estimates
    (train3['hm'] >= 90)
]

con = [
    4, 3, 2, 1
]

train3['fog_risk'] = np.select(cri, con, default = 0)

In [74]:
# 안개 위험군 분류 지표
# test
cri = [
    # 5단계: high risk
    (test2['hm'] >= 97) & (test2['ws10_ms'] <= 7) & (test2['re'] == 0),

    # 4단계: middle risk
    (test2['hm'] < 97) & (test2['hm'] >= 95) & (test2['ws10_ms'] <= 7) & (test2['re'] == 0),

    # 3단계: Low risk
    (test2['hm'] < 95) & (test2['hm'] >= 90) & (test2['ws10_ms'] <= 7) & (test2['re'] == 0),

    # 2단계: Risk not estimates
    (test2['hm'] >= 90)
]

con = [
    4, 3, 2, 1
]

test2['fog_risk'] = np.select(cri, con, default = 0)

#### 4-4) 연도 원상태로 되돌리기

- 연도를 임의로 바꿨기 때문에 다시 되돌리기로 한다.

In [75]:
# 연도 원상태로 되돌리기
cri = [
    train3['year'] == 2020,
    train3['year'] == 2021,
    train3['year'] == 2022,
    train3['year'] == '2020.0',
    train3['year'] == '2021.0',
    train3['year'] == '2022.0',
    train3['year'] == 'I',
    train3['year'] == 'J',
    train3['year'] == 'K',
]

con = [
    'I', 'J', 'K', 'I', 'J', 'K', 'I', 'J', 'K'
]
train3['year'] = np.select(cri, con, default = 2022)

#### 4-5) 풍향

- 0 ~ 360으로 이어지게끔 만들어주기 위해 삼각 치환을 활용한다.

In [77]:
# train
train3['sin_deg'] = np.sin(train3['ws10_deg'] * np.pi / 180)
train3['cos_deg'] = np.cos(train3['ws10_deg'] * np.pi / 180)

In [78]:
# test
test2['sin_deg'] = np.sin(test2['ws10_deg'] * np.pi / 180)
test2['cos_deg'] = np.cos(test2['ws10_deg'] * np.pi / 180)

#### 4-6) ts

- 지면온도가 터무니없이 0으로 귀결되는 값이 train에서 존재하기 때문에 이 값을 대치를 적용해야 한다.
- knn 이후 적용하는 것이므로 후처리를 적용한다.

진행 방법
> 1. 뾰족산 제거
> 2. 웅덩이 같이 이상치가 지속되는 부분 제거

단, 이 과정은 train에 대해서만 적용할 것이며 test에서는 적용하지 않는다 -> dataleakge 고려

In [194]:
# 앞뒤로 값 측정하기
train3['ts_before'] = train3['ts'].shift(1)
train3['ts_after'] = train3['ts'].shift(-1)

In [195]:
# 뾰족산 제거
train3['ts'] = np.where((abs(train3['ts'] - train3['ts_before']) >= 10) & (abs(train3['ts'] - train3['ts_after']) >= 10), round((train3['ts_before']+train3['ts_after'])/2, 1), train3['ts'])

In [196]:
# 제거한 이후 앞의값만 측정하기
train3['ts_before'] = train3['ts'].shift(1)

In [204]:
train3[(3 <= train3['month']) & (train3['month'] <= 11) & (train3['time'] <= 20) & (train3['time'] >= 7) & (abs(train3['ts'] - train3['ts_before']) >= 15)].index

Int64Index([  27279,   27287,   87060,   87361,  437245,  438544,  509250,
             656544,  656548,  656560,  823887,  823890,  917489,  969888,
             969890,  969891, 1077366, 1138548, 1138837, 1138996, 1141854,
            1399154, 1512629, 1555247, 1626672, 1679214, 1679350, 1679354,
            1679790, 1679933, 1753265, 1991711, 1991715, 2061719, 2061721,
            2167083, 2230466, 2241726, 2435374, 2435376, 2515888, 2515890,
            2535146, 2535285, 2535289, 2598091, 2609484, 2692678, 2695164,
            2695166, 2731425, 2731468, 2731492, 2731495, 2747724, 2747726,
            2755937, 2755939, 2808346, 2808351, 2811058, 2811099, 2828637,
            2828640, 2852844, 2852846, 2855854, 2855856, 2914619, 2914621,
            2914622, 2916640],
           dtype='int64')

In [277]:
# 위에 있는 인덱스를 통해 수동으로 골라서 적용하기
start = [27279, 656544, 969888, 1077366, 1138996, 1141844, 1626671, 1679350, 1991711, 2061719, 2435374, 2515888, 2535285, 2692668, 2695164, 2731492, 2747724, 2755937, 2811058, 2811097, 2828637, 2852844, 2855854, 2914619]
end = [27287, 656548, 969892, 1077399, 1138998, 1141854, 1626672, 1679354, 1991715, 2061721, 2435376, 2515890, 2535289, 2692678, 2695166, 2731495, 2747726, 2755939, 2811063, 2811099, 2828640, 2852846, 2855856, 2914622]
front = [35.5, 50.3, 47.10, 22.0, 27.20, 30.00, 9.30, -3.52, 31.0, 7.50, 14.8, 26.0, -2.72, 0.20, -0.30, -2.18, 7.70, 33.30, 23.60, 20.30, 20.30, 8.20, 14.8, 55.4, ]
back = [52.3, 57.4, 41.80, 19.2, 8.40, 32.26, 19.60, -4.90, 28.1, 7.80, 19.8, 18.1, -4.80, -1.50, -0.20, -2.60, 8.60, 30.96, 29.18, 19.80, 21.00, 9.00, 19.8, 45.2, ]

In [285]:
# 값 대입하기
for s, e, f, b in zip(start, end, front, back):
    train3.loc[s:e, 'ts'] = round((f + b) / 2, 1)

In [288]:
# 제거한 이후 앞의값만 측정하기
train3['ts_before'] = train3['ts'].shift(1)

In [291]:
# ts관련 변수 제거
train3.drop(['ts_before', 'ts_after'], axis = 1, inplace = True)

In [292]:
# save the data
train3.to_csv('../data/train_final_preprocess.csv', index = False)
test2.to_csv('../data/test_final_preprocess.csv', index = False)